# The following Class is used to normalize the solar inputs based on the longitude, latitude, cell type and capacity, tilt, asmuth, and the specific day of the year.

In [8]:
import numpy as np

class Preprocess:
    def __init__(self, latitude, longitude):
        """
            Latitude in degrees (+ north, - south)
            Longitude in degrees (+ east, - west)
        """
        self.latitude = latitude
        self.longitude = longitude

    def H_prime(self, day_of_year):
        """
        day_of_year: int (1-365 or 366)
        This is the average solar constant adjusted for the day of the year. 
        It is used to approximate the solar irradiation when there is no clouds and the sun is at a normal angle.
        """
        H0 = 1367 # W/m² (average solar constant)
        H0_prime = H0 * (1 + 0.033 * np.cos(np.radians(360 * day_of_year / 365)))
        return H0_prime
    def estimate_solar_irradiation(self, day_of_year, hour_local, tilt_deg=28, azimuth_deg=220):
        """Estimate solar irradiation (W/m²) on a tilted surface.
        day_of_year: int (1-365 or 366)
        hour_local: local time in hours, e.g., 13.5 = 1:30 PM
        tilt_deg: tilt angle of the panel from horizontal (degrees), 0° = horizontal
        azimuth_deg: azimuth angle of the panel (degrees), 0° = North, 90° = East, 180° = South, 270° = West
        returns the estimated solar irradiation (W/m²)
        """
    
        # Get corrected solar constant
        H0 = self.H_prime(day_of_year)

        # Convert to radians
        phi = np.radians(self.latitude)
        tilt = np.radians(tilt_deg)
        azimuth = np.radians(azimuth_deg)

        # Solar declination angle (radians)
        delta = 23.45 * np.sin(np.radians(360/365 * (day_of_year - 81)))
        delta = np.radians(delta)

        # Equation of Time (minutes)
        # B = 360/365 * (day_of_year - 81) is the angle in degrees
        # EoT = 9.87 * sin(2B) - 7.53 * cos(B) - 1.5 * sin(B)
        # where B is in degrees
        # EoT is the difference between solar time and clock time
        # due to the elliptical shape of the Earth's orbit and axial tilt, 81st day of the year is around March 21st which is the equinox
        B = 360/365 * (day_of_year - 81)
        EoT = 9.87 * np.sin(np.radians(2*B)) - 7.53 * np.cos(np.radians(B)) - 1.5 * np.sin(np.radians(B))

        # Time correction factor (minutes)
        timezone_longitude = round(self.longitude / 15) * 15
        time_correction = 4 * (self.longitude - timezone_longitude) + EoT

        # Solar time (hours)
        solar_time = hour_local + time_correction / 60

        # Hour angle omega (radians)
        omega = np.radians(15 * (solar_time - 12))

        # Cosine of solar zenith angle (for reference)
        cosZ = np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.cos(omega)
        cosZ = max(cosZ, 0)

        if cosZ == 0:
            return 0  # Sun is below horizon, no irradiation

        # Solar azimuth and solar elevation
        sin_alpha = np.sin(phi) * np.sin(delta) + np.cos(phi) * np.cos(delta) * np.cos(omega)  # sin of solar elevation angle
        alpha = np.arcsin(sin_alpha)  # solar elevation angle in radians

        # Solar azimuth angle (γs)
        cos_azimuth = (np.sin(delta) - np.sin(phi) * np.sin(alpha)) / (np.cos(phi) * np.cos(alpha))
        cos_azimuth = np.clip(cos_azimuth, -1, 1)  # numerical safety
        solar_azimuth = np.arccos(cos_azimuth)  # radians

        if omega > 0:  # afternoon
            solar_azimuth = 2*np.pi - solar_azimuth

        # Angle of incidence (θ) between sun rays and panel normal
        cos_theta = (
            np.sin(delta) * np.sin(phi) * np.cos(tilt)
            - np.sin(delta) * np.cos(phi) * np.sin(tilt) * np.cos(azimuth)
            + np.cos(delta) * np.cos(phi) * np.cos(tilt) * np.cos(omega)
            + np.cos(delta) * np.sin(phi) * np.sin(tilt) * np.cos(azimuth) * np.cos(omega)
            + np.cos(delta) * np.sin(tilt) * np.sin(azimuth) * np.sin(omega)
        )
        cos_theta = max(cos_theta, 0)  # No backside radiation

        # Estimate irradiation on tilted panel
        irradiation = H0 * cos_theta

        return irradiation


    def normalize(self, cell_type, num_panels, tilt_angle_deg, azimuth_deg, total_rated_capacity_kw):
        """
        Normalize the irradiation based on panel setup and system specs.

        Args:
            cell_type (str): Type of solar cell ('mono', 'poly', 'thinfilm')
            num_panels (int): Number of panels
            tilt_angle_deg (float): Tilt of the panels (degrees)
            azimuth_deg (float): Orientation of the panels (degrees from North)
            total_rated_capacity_kw (float): Total rated system capacity (kW)

        Returns:
            float: Normalized irradiation value (W/m²)
        """
        # Define typical efficiencies by cell type
        efficiencies = {
            'mono': 0.20,
            'poly': 0.17,
            'thinfilm': 0.11
        }

        # Get efficiency based on cell type
        cell_type = cell_type.lower()
        if cell_type not in efficiencies:
            raise ValueError(f"Unknown cell type '{cell_type}'. Choose from {list(efficiencies.keys())}.")
        
        efficiency = efficiencies[cell_type]

        # Calculate total panel area (in m²)
        total_rated_capacity_w = total_rated_capacity_kw * 1000  # convert kW to W
        total_area_m2 = total_rated_capacity_w / (efficiency * 1000)  # 1000 W/m² at STC

        # Irradiance normalized per m²
        normalized_irradiation = (total_rated_capacity_w / total_area_m2)  # Should give back 1000 W/m² ideally

        # Could include tilt and azimuth correction here later if needed

        return normalized_irradiation


# Create a Preprocess object for San Francisco
pre = Preprocess(latitude=37.7749, longitude=-122.4194)

# Estimate solar irradiation on June 21st at 2:00 PM local time
day_of_year = 172
hour_local = 14.0

irradiation = pre.estimate_solar_irradiation(day_of_year, hour_local)
print(f"Estimated solar irradiation: {irradiation:.2f} W/m²")
# Example usage:
irradiance = pre.estimate_solar_irradiation(
    day_of_year=150, hour_local=14, tilt_deg=30, azimuth_deg=180
)
print(f"Irradiance on tilted panel: {irradiance:.2f} W/m²")


Estimated solar irradiation: 782.16 W/m²
Irradiance on tilted panel: 867.07 W/m²
